In [83]:
##get data
import numpy as np
import sys
sys.path
sys.path.append('../models')
import pickle
import math
import tensorflow as tf
import gpflow
import random

In [2]:
##set grid
u1 = np.arange(.1,1.1,.1)
u2 = np.arange(.1,1.1,.1)
rho = np.arange(.1,1.1,.2)

In [3]:
v1 = np.arange(1,10,1)
v2 = np.arange(10,10,1)

In [4]:
noise = np.arange(.1,1.1,.1)

In [5]:
with open('../../Downloads/processed_data_for_gridsearch_GP.pkl','rb') as f:
    data = pickle.load(f)

In [6]:
X=data[0]
y=data[1]

In [7]:
def rbf_custom_np( X, X2=None):
        #print(X)
        #print(X2)
        if X2 is None:
             X2=X
        return math.exp(-((X-X2)**2)/1.0)
        #tf.divide(tf.square(-tf.subtract(X,X2)),tf.constant(1.0,dtype=tf.float64)))


In [8]:
import CustomKernelStatic

In [92]:
def get_likelihood(u1,u2,s1,s2,rho,noise,X,y,rhos,users):
    sess = tf.Session()
    k =  CustomKernelStatic.CustomKernelStatic(20,mysession=sess,rhos=rhos,select_users=users,baseline_indices=[i for i in range(18)],\
                  psi_indices=[0,6],user_day_index=19,user_index=18,num_data_points=len(X),initial_u1=u1,initial_u2=u2,\
                           initial_s1=s1,initial_s2=s2,initial_rho=rho,initial_noise=noise)
  
    m = gpflow.models.GPR(X,y, kern=k)
    m.initialize(session=sess)
 
    try:
        l =m.compute_log_likelihood()
        print('huzzah')
        print(l)
    except:
        l=-100
        print('not huzzah not')
        print(l)
    sess.close()
    return l
    ##get likelihood #return likelihood

In [94]:
def run():
    to_save = {}
    u1 = np.arange(.1,1.1,.1)
    u2 = np.arange(.1,1.1,.1)
    first_rhos = np.arange(.1,1.1,.2)
    v1 = np.arange(1,10,1)
    v2 = np.arange(1,10,1)
    noise = np.arange(.1,1.1,.1)
    
    with open('../../Downloads/processed_data_for_gridsearch_GP.pkl','rb') as f:
        data = pickle.load(f)
    
    X=data[0]
    y=data[1]
    for x in X:
        x[0]=1+random.random()/.01
    for x in X:
        x[6]=1+random.random()/.01
    rdayone = [x[19] for x in X]
    rdaytwo = rdayone
    rhos = np.array([[rbf_custom_np( rdayone[i], X2=rdaytwo[j]) for j in range(len(X))] for i in range(len(X))])
    users = np.array([[float(X[i][18]==X[j][18]) for j in range(len(X))] for i in range(len(X))])
    
    print(X.shape)
    print(y.shape)
    print(len(set(X[:,0])))
    for uone in u1:
        for utwo in u2:
            for rho in first_rhos:
                for s1 in v1:
                    for s2 in v2:
                        for n in noise:
                            likli=get_likelihood(uone,uone,s1,s2,rho,n,X,y,rhos,users)
                             
                            key = '-'.join([str(i) for i in[uone,utwo,rho,s1,s2,n]])
                            to_save[key]=likli
                            
    return to_save

In [ ]:
 run()
    

(6286, 20)
(6286, 1)
6286


not huzzah not
-100


In [11]:
    rdayone = [x[19] for x in X]
    rdaytwo = rdayone
    rhos = np.array([[rbf_custom_np( rdayone[i], X2=rdaytwo[j]) for j in range(len(X))] for i in range(len(X))])
    users = np.array([[float(X[i][18]==X[j][18]) for j in range(len(X))] for i in range(len(X))])
    

In [12]:
    sess = tf.Session()
    k =  CustomKernelStatic.CustomKernelStatic(20,mysession=sess,rhos=rhos,select_users=users,baseline_indices=[i for i in range(18)],\
                  psi_indices=[0,7],user_day_index=19,user_index=18,num_data_points=len(X),initial_u1=u1[0],initial_u2=u2[0],\
                           initial_s1=v1[0],initial_s2=v2[0],initial_rho=rho[0],initial_noise=noise[0])
  
    m = gpflow.models.GPR(X,y, kern=k)
    m.initialize(session=sess)

In [13]:
lik=m.compute_log_likelihood()

InvalidArgumentError: Cholesky decomposition was not successful. The input might not be valid.
	 [[node GPR-f44b59b5-5/likelihood_1/Cholesky (defined at /Users/sabina/anaconda3/envs/py36/lib/python3.6/site-packages/gpflow/models/gpr.py:64)  = Cholesky[T=DT_DOUBLE, _device="/job:localhost/replica:0/task:0/device:CPU:0"](GPR-f44b59b5-5/likelihood_1/add_3)]]

Caused by op 'GPR-f44b59b5-5/likelihood_1/Cholesky', defined at:
  File "/Users/sabina/anaconda3/envs/py36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/sabina/anaconda3/envs/py36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/sabina/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/sabina/anaconda3/envs/py36/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/sabina/anaconda3/envs/py

In [46]:
temp_x = np.array([x[1:10] for x in X[:14]])

In [47]:
np.linalg.cholesky(temp_x)

LinAlgError: Last 2 dimensions of the array must be square

In [48]:
temp_x

array([[1.        , 0.        , 1.        , 1.        , 0.        ,
        0.        , 1.        , 0.        , 1.        ],
       [1.        , 0.        , 0.66666667, 1.        , 0.        ,
        0.        , 1.        , 0.        , 0.66666667],
       [1.        , 0.        , 0.33333333, 1.        , 0.        ,
        0.        , 1.        , 0.        , 0.33333333],
       [1.        , 0.        , 0.33333333, 1.        , 0.        ,
        0.        , 1.        , 0.        , 0.33333333],
       [1.        , 0.        , 0.66666667, 1.        , 0.        ,
        0.        , 1.        , 0.        , 0.66666667],
       [1.        , 1.        , 0.66666667, 1.        , 0.        ,
        0.        , 1.        , 1.        , 0.66666667],
       [1.        , 1.        , 1.        , 1.        , 1.        ,
        0.        , 1.        , 1.        , 1.        ],
       [1.        , 0.        , 1.        , 1.        , 1.        ,
        0.        , 1.        , 0.        , 1.        ],


In [49]:
for x in X:
    x[0]=1



In [59]:
len(set(X[:,1]))

2

In [91]:
for i in range(len(X[0])):
    print(i)
    print(len(set(X[:,i])))
    print('\n')

0
1


1
2


2
2


3
4


4
2


5
2


6
1


7
2


8
2


9
4


10
2


11
2


12
2


13
3


14
3


15
7


16
3


17
3


18
36


19
62


